<a href="https://colab.research.google.com/github/efecte8/funcraft/blob/main/funcraft_backend_3_1_sdxl_lcm_lora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

make sure to run on gpu runtime

license



In [ ]:
#Copyright 2023 funcraft

#Licensed under the Apache License, Version 2.0 (the "License");
#you may not use this file except in compliance with the License.
#You may obtain a copy of the License at

#    http://www.apache.org/licenses/LICENSE-2.0

#Unless required by applicable law or agreed to in writing, software
#distributed under the License is distributed on an "AS IS" BASIS,
#WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#See the License for the specific language governing permissions and
#limitations under the License.

<figure>
<center>
<iframe src="https://drive.google.com/file/d/1E9DbsSF9nQ-KohIXUcE3I23DcYaDj54E/preview" width="640" height="480" allow="autoplay"></iframe>
</figure>


![Steps](https://i.imgur.com/cld20dj.jpg)

#Install requirements ~1min


*   APP API - FLASK, NGROK (tunnel to local)
*   ML - DIFFUSERS, TRANSFORMERS, ACCELERATE, PEFT



ML dependencies

In [ ]:
!pip install --quiet --upgrade diffusers transformers accelerate peft
!pip install --quiet flask
!pip install --quiet flask-ngrok pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 20.5 MB/s eta 0:00:00


In [ ]:
#!ngrok config add-authtoken xxxxxx

# Model Pipeline:  sdxl1.0 LCM Lora ~2min

In [ ]:
import torch
from diffusers import LCMScheduler, DiffusionPipeline, AutoPipelineForText2Image, AutoPipelineForInpainting, AutoPipelineForImage2Image

base_pipe = DiffusionPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    variant="fp16",
    torch_dtype=torch.float16
).to("cuda")

# set scheduler
base_pipe.scheduler = LCMScheduler.from_config(base_pipe.scheduler.config)

# load LoRAs
base_pipe.load_lora_weights("latent-consistency/lcm-lora-sdxl", adapter_name="lcm")
base_pipe.fuse_lora()
generator = torch.manual_seed(0)
base_pipe.generator = generator



The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

model_index.json:   0%|          | 0.00/609 [00:00<?, ?B/s]

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

tokenizer/merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

...

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

diffusion_pytorch_model.fp16.safetensors:   0%|          | 0.00/167M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

The config attributes {'skip_prk_steps': True} were passed to LCMScheduler, but are not expected and will be ignored. Please verify your scheduler_config.json configuration file.


pytorch_lora_weights.safetensors:   0%|          | 0.00/394M [00:00<?, ?B/s]

In [ ]:
img2img_pipe= AutoPipelineForImage2Image.from_pipe(base_pipe)
img2img_pipe.to("cuda")

StableDiffusionXLImg2ImgPipeline {
  "_class_name": "StableDiffusionXLImg2ImgPipeline",
  "_diffusers_version": "0.24.0",
  "_name_or_path": "stabilityai/stable-diffusion-xl-base-1.0",
  "feature_extractor": [
    null,
    null
  ],
  "force_zeros_for_empty_prompt": true,
  "image_encoder": [
    null,
    null
  ],
  "requires_aesthetics_score": false,
  "scheduler": [
    "diffusers",
    "LCMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "text_encoder_2": [
    "transformers",
    "CLIPTextModelWithProjection"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "tokenizer_2": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [ ]:
inpainting_pipe = AutoPipelineForInpainting.from_pipe(base_pipe)
inpainting_pipe.to("cuda")

StableDiffusionXLInpaintPipeline {
  "_class_name": "StableDiffusionXLInpaintPipeline",
  "_diffusers_version": "0.24.0",
  "_name_or_path": "stabilityai/stable-diffusion-xl-base-1.0",
  "feature_extractor": [
    null,
    null
  ],
  "force_zeros_for_empty_prompt": true,
  "image_encoder": [
    null,
    null
  ],
  "requires_aesthetics_score": false,
  "scheduler": [
    "diffusers",
    "LCMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "text_encoder_2": [
    "transformers",
    "CLIPTextModelWithProjection"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "tokenizer_2": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [ ]:
#text to image function, check for generator as well for seed
def text_to_image(prompt, negative_prompt, num_inference_steps=4, guidance_scale=1):
  image0 = base_pipe(prompt=prompt, negative_prompt=negative_prompt, guidance_scale=guidance_scale,num_inference_steps=num_inference_steps ).images[0]
  torch.cuda.empty_cache()
  return image0

#image to image function, check for generator as well for seed
def image_to_image(prompt, negative_prompt, image, strength=0.8, num_inference_steps=4, guidance_scale=1):
  image1=img2img_pipe(prompt, negative_prompt=negative_prompt, image=image, strength=strength, guidance_scale=guidance_scale, num_inference_steps=num_inference_steps).images[0]
  torch.cuda.empty_cache()
  return image1

#inpainting
def image_inpainting(prompt, negative_prompt, image, mask_image, strength=0.7, num_inference_steps=4, guidance_scale=1):
  image2= inpainting_pipe(prompt=prompt, negative_prompt=negative_prompt, image=image, mask_image=mask_image, strength=strength, num_inference_steps=num_inference_steps, guidance_scale=guidance_scale).images[0]
  torch.cuda.empty_cache()
  return image2

## Styles

In [ ]:
style_dictionary = {

       "Photorealistic": {
        "prompt": ", photorealistic, hyperrealistic, ultrarealistic, photography, dslr, highly detailed, 8k, sharp focus, 35mm photograph",
        "neg_prompt": "drawing, painting, crayon, sketch, graphite, impressionist, noisy, blurry, soft, deformed"
    },


        "AnalogFilm": {
        "prompt": ", analog film photo {prompt}. faded film, desaturated, 35mm photo, grainy, vignette, vintage, Kodachrome, Lomography, stained, highly detailed, found footage",
        "neg_prompt": "painting, drawing, illustration, glitch, deformed, mutated, cross-eyed, ugly, disfigured"
    },


        "Portrait": {
        "prompt": ", portrait, highly detailed",
        "neg_prompt": "glitch, deformed, mutated, ugly, disfigured"
    },


        "Futuristic": {
        "prompt": ", futuristic, inspired by Krenz Cushart, neoism, kawacy, wlop, gits anime",
        "neg_prompt": ""
    },

         "Digital painting artwork": {
        "prompt": ", digital artwork, illustrative, painterly, matte painting, highly detailed, cinematic composition",
        "neg_prompt": "photo, photorealistic, realism, ugly"
    },

        "Digital painting Mixed Media": {
        "prompt": ", a digital painting, by Jason Benjamin, shutterstock, colorful vector illustration, mixed media style illustration, epic full color illustration, mascot illustration",
        "neg_prompt": ""
    },


       "Cyberpunk": {
        "prompt": ", cyberpunk",
        "neg_prompt": ""
    },


        "Highly detailed illustration": {
        "prompt": ",inspired by Cyril Rolando, shutterstock, highly detailed illustration, full color illustration, very detailed illustration, dan mumford and alex grey style",
        "neg_prompt": ""
    },

        "Chinese Painting": {
        "prompt": ", a fine art painting, by Qiu Ying, no gradients, flowing sakura silk, beautiful oil painting",
        "neg_prompt": ""
    },

        "Anime": {
        "prompt": "anime artwork {prompt}. anime style, key visual, vibrant, studio anime, highly detailed",
        "neg_prompt": "photo, deformed, black and white, realism, disfigured, low contrast"
    },

        "Fantasy": {
        "prompt": "ethereal fantasy concept art of {prompt}. magnificent, celestial, ethereal, painterly, epic, majestic, magical, fantasy art, cover art, dreamy",
        "neg_prompt": "photographic, realistic, realism, 35mm film, dslr, cropped, frame, text, deformed, glitch, noise, noisy, off-center, deformed, cross-eyed, closed eyes, bad anatomy, ugly, disfigured, sloppy, duplicate, mutated, black and white"
    },


        "Cinematography": {
        "prompt": "cinematic film still {prompt}. shallow depth of field, vignette, highly detailed, high budget Hollywood movie, bokeh, cinemascope, moody",
        "neg_prompt": "anime, cartoon, graphic, text, painting, crayon, graphite, abstract, glitch, deformed, mutated, ugly, disfigured"
    },


        "Synthwave": {
        "prompt": "vaporwave synthwave style {prompt}. cyberpunk, neon, vibes, stunningly beautiful, crisp, detailed, sleek, ultramodern, high contrast, cinematic composition",
        "neg_prompt": "illustration, painting, crayon, graphite, abstract, glitch, deformed, mutated, ugly, disfigured"
    },

        "Isometric": {
        "prompt": "isometric style {prompt}. vibrant, beautiful, crisp, detailed, ultra detailed, intricate",
        "neg_prompt": "deformed, mutated, ugly, disfigured, blur, blurry, noise, noisy, realistic, photographic"
    },

        "Low Poly Game Art": {
        "prompt": "low-poly style {prompt}. ambient occlusion, low-poly game art, polygon mesh, jagged, blocky, wireframe edges, centered composition",
        "neg_prompt": "noisy, sloppy, messy, grainy, highly detailed, ultra textured, photo"
    },


        "Clay Art": {
        "prompt": ", claymation style {prompt}. sculpture, clay art, centered composition, play-doh",
        "neg_prompt": "sloppy, messy, grainy, highly detailed, ultra textured, photo, mutated"
    },

        "3D Model": {
        "prompt": "professional 3d model {prompt} . octane render, highly detailed, volumetric, dramatic lighting",
        "neg_prompt": "ugly, deformed, noisy, low poly, blurry, painting"
    },


        "Origami": {
        "prompt": "origami style {prompt}. paper art, pleated paper, folded, origami art, pleats, cut and fold, centered composition",
        "neg_prompt": "noisy, sloppy, messy, grainy, highly detailed, ultra textured"
    },


        "Line Art": {
        "prompt": "line art drawing {prompt}. professional, sleek, modern, minimalist, graphic, line art, vector graphics",
        "neg_prompt": "anime, photorealistic, 35mm film, deformed, glitch, blurry, noisy, off-center, deformed, cross-eyed, closed eyes, bad anatomy, ugly, disfigured, mutated, realism, realistic, impressionism, expressionism, oil, acrylic"
    },


        "PixelArt": {
        "prompt": "pixelart {prompt}, pixel art, detailed",
        "neg_prompt": ""
    },



        "Fabl": {
        "prompt": ", award winning photography, Elke vogelsang",
        "neg_prompt": ""
    }


    }

In [ ]:
def style_modifier(input_prompt, input_neg_prompt, style):
  style_prompt= style_dictionary[style]['prompt']
  if "{prompt}" in style_prompt:
    output_prompt = style_prompt.replace("{prompt}", input_prompt)
  else:
    output_prompt= input_prompt + style_dictionary[style]['prompt']

  if input_neg_prompt =='':
    output_neg_prompt=style_dictionary[style]['neg_prompt']
  else:
    output_neg_prompt= input_neg_prompt + ", " + style_dictionary[style]['neg_prompt']

  return output_prompt, output_neg_prompt

In [ ]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://w22g5cdowoi-496ff2e9c6d22116-5000-colab.googleusercontent.com/


#APP and API
- create a web app backend that sends the images to the local using a ngroc url
- create the api routes
- run the app and start listening for requests

In [ ]:
from flask import Flask, request, send_file
from PIL import Image
import io
import zipfile

from flask_ngrok import run_with_ngrok
from pyngrok import ngrok

app = Flask(__name__)
public_url = ngrok.connect(5000)
run_with_ngrok(app)  # Start ngrok when app is run

@app.route('/inpainting', methods=['POST'])
def process_image2():
    prompt = str(request.form['prompt'])
    gen_mode = int(request.form['gen_mode'])
    guidance_scale = int(request.form['guidance_scale'])
    strength_scale = float(request.form['strength_scale'])
    number_of_steps = int(request.form['number_of_steps'])
    seed = int(request.form['seed'])
    negative_prompt= str(request.form['negative_prompt'])

    print(f'Prompt: {prompt}')
    print(f'Gen Mode: {gen_mode}')
    print(f'Guidance Scale: {guidance_scale}')
    print(f'Strength Scale: {strength_scale}')
    print(f'Number of Steps: {number_of_steps}')
    print(f'Seed: {seed}')
    print(f'Negative prompt: {negative_prompt}')

    image_file = request.files['image']  # Get the uploaded image file
    mask_image_file = request.files['mask_image']

    # Convert the image files to PIL Image objects
    image = Image.open(image_file.stream)
    mask_image = Image.open(mask_image_file.stream)
    #display(image)
    #display(mask_image)
    image_inpaint= image_inpainting(prompt=prompt, negative_prompt=negative_prompt, image=image, mask_image=mask_image, strength=strength_scale, num_inference_steps=number_of_steps, guidance_scale=guidance_scale)
    #display(image_inpaint)

    # Convert the PIL Image to bytes
    image_inpaint_bytes = io.BytesIO()
    image_inpaint.save(image_inpaint_bytes, format='PNG')
    image_inpaint_bytes.seek(0)

    # Return the processed image as a response
    return send_file(image_inpaint_bytes, mimetype='image/png')

@app.route('/imagetoimage', methods=['POST'])
def process_image1():
    prompt = str(request.form['prompt'])
    gen_mode = int(request.form['gen_mode'])
    guidance_scale = int(request.form['guidance_scale'])
    strength_scale = float(request.form['strength_scale'])
    number_of_steps = int(request.form['number_of_steps'])
    seed = int(request.form['seed'])
    negative_prompt= str(request.form['negative_prompt'])
    selected_style=str(request.form['selected_style'])

    print(f'Prompt: {prompt}')
    print(f'Gen Mode: {gen_mode}')
    print(f'Guidance Scale: {guidance_scale}')
    print(f'Strength Scale: {strength_scale}')
    print(f'Number of Steps: {number_of_steps}')
    print(f'Seed: {seed}')
    print(f'Negative prompt: {negative_prompt}')
    print(f'Selected style : {selected_style}')

    if selected_style != 'No Style':
      prompt, negative_prompt= style_modifier(input_prompt=prompt, input_neg_prompt=negative_prompt, style=selected_style)



    image_file = request.files['image']  # Get the uploaded image file

    # Convert the image files to PIL Image objects
    image = Image.open(image_file.stream)
    #display(image)
    image_imagetoimage= image_to_image(prompt=prompt, negative_prompt=negative_prompt, image=image, strength=strength_scale, num_inference_steps=number_of_steps, guidance_scale=guidance_scale)
    #display(image_imagetoimage)

    # Convert the PIL Image to bytes
    imagetoimage_bytes = io.BytesIO()
    image_imagetoimage.save(imagetoimage_bytes, format='PNG')
    imagetoimage_bytes.seek(0)

    # Return the processed image as a response
    return send_file(imagetoimage_bytes, mimetype='image/png')


@app.route('/texttoimage', methods=['POST'])
def process_image0():
    prompt = str(request.form['prompt'])
    gen_mode = int(request.form['gen_mode'])
    guidance_scale = int(request.form['guidance_scale'])
    strength_scale = float(request.form['strength_scale'])
    number_of_steps = int(request.form['number_of_steps'])
    seed = int(request.form['seed'])
    negative_prompt= str(request.form['negative_prompt'])
    selected_style=str(request.form['selected_style'])

    print(f'Prompt: {prompt}')
    print(f'Gen Mode: {gen_mode}')
    print(f'Guidance Scale: {guidance_scale}')
    print(f'Strength Scale: {strength_scale}')
    print(f'Number of Steps: {number_of_steps}')
    print(f'Seed: {seed}')
    print(f'Negative prompt: {negative_prompt}')
    print(f'Selected style : {selected_style}')

    if selected_style != 'No Style':
      prompt, negative_prompt= style_modifier(input_prompt=prompt, input_neg_prompt=negative_prompt, style=selected_style)


    image_texttoimage= text_to_image(prompt=prompt, negative_prompt=negative_prompt, num_inference_steps=number_of_steps, guidance_scale=guidance_scale)
    #display(image_texttoimage)

    # Convert the PIL Image to bytes
    image_texttoimage_bytes = io.BytesIO()
    image_texttoimage.save(image_texttoimage_bytes, format='PNG')
    image_texttoimage_bytes.seek(0)

    # Return the processed image as a response
    return send_file(image_texttoimage_bytes, mimetype='image/png')



if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://05a8-35-193-147-163.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
Prompt: cat working in a art studio
Gen Mode: 0
Guidance Scale: 1
Strength Scale: 0.6
Number of Steps: 4
Seed: 0
Negative prompt: 
Selected style : No Style


  0%|          | 0/4 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [19/Dec/2023 14:41:34] "POST /texttoimage HTTP/1.1" 200 -


...

  0%|          | 0/5 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [19/Dec/2023 14:51:45] "POST /imagetoimage HTTP/1.1" 200 -


Prompt: cat working in a art studio on a painting on canvas
Gen Mode: 2
Guidance Scale: 1
Strength Scale: 0.5
Number of Steps: 10
Seed: 0
Negative prompt: 


  0%|          | 0/5 [00:00<?, ?it/s]

INFO:werkzeug:127.0.0.1 - - [19/Dec/2023 14:52:33] "POST /inpainting HTTP/1.1" 200 -
